![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and MCP server to invoke on AutoAI deployment.

This notebook contains the steps and code to demonstrate support watsonx with MCP service.

# Set up the environment

Install and import dependecies

In [ ]:
%pip install langchain-ibm
%pip install langchain
%pip install langgraph
%pip install python-dotenv
%pip install mcp
%pip install langchain_mcp_adapters

## ChatWatsonx definition

Create an instance of `ChatWatsonx` from environment variable.

In [2]:
from utils import prepare_chat_watsonx

chat_watsonx = prepare_chat_watsonx()

### Async Agent Runner: Initializing LangChain Agent with MCP Tools

The run_agent(question, model) function is an asynchronous workflow that sets up and runs a LangChain agent using tools loaded from an MCP (Modular Capability Provider) server. It performs the following steps:

- Connects to a local server via Server-Sent Events (SSE) to enable real-time communication.
- Initializes a client session to interact with the MCP system.
- Loads external tools (APIs, functions, etc.) available from the MCP environment.
- Creates a ReAct Agent using the given instance of Chat Watsonx and the loaded tools.
- Sends a question (prompt) to the agent and asynchronously waits for its response.
- Returns the final answer generated by the agent.

In [3]:
from mcp import ClientSession
from mcp.client.sse import sse_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

# Define the function
async def run_agent(question, chat_watsonx):
    async with sse_client(url="http://127.0.0.1:8000/sse") as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # Load tools
            tools = await load_mcp_tools(session)

            # Create agent
            agent = create_react_agent(chat_watsonx, tools)

            # Run the agent
            agent_response = await agent.ainvoke({"messages": question})
            
            return agent_response

## Math Questions

We define a mathematical question that the agent should solve by selecting the appropriate tool.

In [4]:
question = "What is 1 + 100?"

In [5]:
result = await run_agent(question, chat_watsonx)

for el in result["messages"]:
    el.pretty_print()

================================ Human Message =================================

What is 1 + 100?
================================== Ai Message ==================================
Tool Calls:
  add (InKbjWca0)
 Call ID: InKbjWca0
  Args:
    a: 1
    b: 100
================================= Tool Message =================================
Name: add

101
================================== Ai Message ==================================

 The result of 1 + 100 is: 101


In [6]:
question = "What is 1 - 100?"

In [7]:
result = await run_agent(question, chat_watsonx)

for el in result["messages"]:
    el.pretty_print()

================================ Human Message =================================

What is 1 - 100?
================================== Ai Message ==================================
Tool Calls:
  sub (tADno2Bi3)
 Call ID: tADno2Bi3
  Args:
    a: 100
    b: 1
================================= Tool Message =================================
Name: sub

99
================================== Ai Message ==================================

 99


## No Risk question

We define a question about a user who should NOT have a credit risk, which the agent is supposed to solve by selecting the appropriate tool.

In [8]:
query = '''Please answer, based on the each informations below about the person whether he or she has credit risk.

Marek is a 35-year-old man applying for a loan. His checking account status shows a small positive balance. He is requesting a loan to be repaid over a duration of 24 months. Marek has a good credit history, having previously repaid all loans successfully. The purpose of the loan is to purchase a new car. The loan amount he is applying for is 8,000 EUR. He has moderate existing savings in his savings account. Marek has been employed for over 4 years at his current job. He plans to allocate 20% of his monthly income to cover the loan installments.
Marek is male and is applying for the loan individually (no others on the loan). He has been living at his current residence for 5 years. He owns a house in the city. Marek’s age is 35. He has no other installment plans ongoing. His housing situation is stable, as he owns his property. He currently has one existing credit with another financial institution.
Marek works as a skilled employee. He has one dependent child. He also owns a telephone. Lastly, Marek is a domestic worker, not a foreign worker.

Returns a simplified message with risk status ('Risk exists' or 'No Risk exist') and probability percentage.
'''

In [9]:
result = await run_agent(query, chat_watsonx)

for el in result["messages"]:
    el.pretty_print()

================================ Human Message =================================

Please answer, based on the each informations below about the person whether he or she has credit risk.

Marek is a 35-year-old man applying for a loan. His checking account status shows a small positive balance. He is requesting a loan to be repaid over a duration of 24 months. Marek has a good credit history, having previously repaid all loans successfully. The purpose of the loan is to purchase a new car. The loan amount he is applying for is 8,000 EUR. He has moderate existing savings in his savings account. Marek has been employed for over 4 years at his current job. He plans to allocate 20% of his monthly income to cover the loan installments.
Marek is male and is applying for the loan individually (no others on the loan). He has been living at his current residence for 5 years. He owns a house in the city. Marek’s age is 35. He has no other installment plans ongoing. His housing situation is stable

## Risk question

We define a question about a user who should have a credit risk, which the agent is supposed to solve by selecting the appropriate tool.

In [10]:
query = '''Please answer, based on the each informations below about the person whether he or she has credit risk.

A 49-year-old male is applying for a 7,138 EUR loan over 35 months to purchase appliances.
He has no checking account and an outstanding credit history, indicating unpaid debts.
Despite over 7 years of employment in a skilled job and savings between 500–1000 EUR, he has two existing credits and a relatively high installment burden (5% of income).
He lives in free housing with a co-applicant, has two dependents, owns a telephone, and has been residing at his current address for 4 years.
Overall, the applicant presents a moderate to high credit risk.

Returns a simplified message with risk status ('Risk exists' or 'No Risk exist') and probability percentage.
'''

In [11]:
result = await run_agent(query, chat_watsonx)

for el in result["messages"]:
    el.pretty_print()

================================ Human Message =================================

Please answer, based on the each informations below about the person whether he or she has credit risk.

A 49-year-old male is applying for a 7,138 EUR loan over 35 months to purchase appliances.
He has no checking account and an outstanding credit history, indicating unpaid debts.
Despite over 7 years of employment in a skilled job and savings between 500–1000 EUR, he has two existing credits and a relatively high installment burden (5% of income).
He lives in free housing with a co-applicant, has two dependents, owns a telephone, and has been residing at his current address for 4 years.
Overall, the applicant presents a moderate to high credit risk.

Returns a simplified message with risk status ('Risk exists' or 'No Risk exist') and probability percentage.

================================== Ai Message ==================================
Tool Calls:
  invoke_credit_risk_deployemnt (GQbjNqhVY)
 Call ID: 